# Import Libraries

Include all necessary modules for analysis.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import folium

# Read-In Data

We read the data in using the Pandas module for easy conversion from a CSV file format to an in-memory DataFrame.

Dataset from: http://insideairbnb.com/vancouver

In [3]:
airbnb_raw = pd.read_csv("../data/vancouver-listings.csv")
pd.set_option('display.max_columns', None)
airbnb_raw.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,13358,https://www.airbnb.com/rooms/13358,20230314070648,2023-03-14,city scrape,Designer Styled One Bedroom Downtown Vancouver,<b>The space</b><br />This suites central loca...,NaN,https://a0.muscache.com/pictures/40034c18-0837...,52116,https://www.airbnb.com/users/show/52116,Lynn,2009-11-07,"Vancouver, Canada",I am from Vancouver and in my free time enjoy ...,within an hour,100%,96%,t,https://a0.muscache.com/im/pictures/user/97e43...,https://a0.muscache.com/im/pictures/user/97e43...,Downtown Vancouver,1,1,"['email', 'phone']",t,t,NaN,West End,NaN,49.28201,-123.12669,Entire condo,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Freezer"", ""Heating"", ""Toaster"", ""Extra pillo...",$157.00,2,90,1,2,90,90,2.0,90.0,NaN,t,18,22,34,212,2023-03-14,443,10,1,2010-06-22,2023-02-24,4.68,4.75,4.80,4.84,4.80,4.92,4.68,22-311727,f,1,1,0,0,2.86
1,13490,https://www.airbnb.com/rooms/13490,20230314070648,2023-03-14,city scrape,Vancouver's best kept secret,This apartment rents for one month blocks of t...,"In the heart of Vancouver, this apartment has ...",https://a0.muscache.com/pictures/73394727/79d5...,52467,https://www.airbnb.com/users/show/52467,Iris,2009-11-08,"Vancouver, Canada",Hello! My husband Fred and I are actors living...,within an hour,100%,91%,t,https://a0.muscache.com/im/users/52467/profile...,https://a0.muscache.com/im/users/52467/profile...,Kensington-Cedar Cottage,3,5,"['email', 'phone']",t,t,"Vancouver, British Columbia, Canada",Kensington-Cedar Cottage,NaN,49.25622,-123.06607,Entire rental unit,Entire home/apt,2,NaN,1 bath,1.0,1.0,"[""Freezer"", ""Books and reading material"", ""Pri...",$150.00,30,180,30,30,180,180,30.0,180.0,NaN,t,0,5,6,152,2023-03-14,96,4,0,2011-06-02,2022-11-06,4.92,4.93,4.96,4.96,4.97,4.78,4.88,NaN,f,1,1,0,0,0.67
2,14267,https://www.airbnb.com/rooms/14267,20230314070648,2023-03-14,previous scrape,EcoLoft Vancouver,"The Ecoloft is located in the lovely, family r...",We live in the centre of the city of Vancouver...,https://a0.muscache.com/pictures/3646de9b-934e...,56030,https://www.airbnb.com/users/show/56030,Peter & Alison,2009-11-20,"Vancouver, Canada",We moved into this central Vancouver location ...,NaN,NaN,NaN,f,https://a0.muscache.com/im/pictures/user/ab0a8...,https://a0.muscache.com/im/pictures/user/ab0a8...,Kensington-Cedar Cottage,1,1,"['email', 'phone']",t,t,"Vancouver, British Columbia, Canada",Kensington-Cedar Cottage,NaN,49.24922,-123.08139,Entire home,Entire home/apt,4,NaN,1 bath,1.0,2.0,"[""Heating"", ""Smoke alarm"", ""Stove"", ""Coffee ma...",$150.00,3,7,3,3,1125,1125,3.0,1125.0,NaN,t,0,0,0,0,2023-03-14,34,0,0,2010-10-03,2021-07-14,4.76,4.84,4.68,4.90,4.68,4.77,4.71,21-156500,t,1,1,0,0,0.22
3,1

# Data Preprocessing

Check integrity of the dataset.

In [ ]:
# Get dataset dimensions
airbnb_raw.shape

In [ ]:
pd.set_option('display.max_rows', None)

rows, _ = airbnb_raw.shape
percentages = round((airbnb_raw.isnull().sum()/rows)*100, 1) # create percentage column for better understanding of the proportion of missing values
pd.DataFrame(
    data = {
    'Missing Values': airbnb_raw.isnull().sum(),
    'Percentage (%)': percentages
    },
    index = airbnb_raw.columns, 
)

It is immediately clear that some features are completely missing their values. We will remove these columns, as well as remove or impute columns with more than 30% of values missing. Columns with missing values of < 5% will also be removed since this dataset is fairly large and will not be significantly affected by this change.

## Dimensionality Reduction

### Feature Selection

As there are more than 70 features, we will select the ones that may be of interest to us in order to reduce the dimensionality of the dataset.

We will not be selecting information that pertains only to the host, as most of it will not be necessary in our analysis.

In [ ]:
# Select features
airbnb = airbnb_raw.loc[:, 
                        ['id', 
                        'host_id',
                        'listing_url', 
                        'last_scraped', 
                        'name', 
                        'description', 
                        'neighbourhood_cleansed', 
                        'latitude',
                        'longitude',
                        'property_type',
                        'room_type',
                        'accommodates',
                        'bathrooms_text',
                        'bedrooms',
                        'beds',
                        'amenities',
                        'price',
                        'minimum_nights',
                        'maximum_nights',
                        'availability_365',
                        'number_of_reviews',
                        'first_review',
                        'review_scores_rating',
                        'calculated_host_listings_count']
]

### Handling Missing Values

In [ ]:
# Show working set features
rows, _ = airbnb.shape
percentages = round((airbnb.isnull().sum()/rows)*100, 1) # create percentage column for better understanding of the proportion of missing values
pd.DataFrame(
    data = {
    'Missing Values': airbnb.isnull().sum(),
    'Percentage (%)': percentages
    },
    index = airbnb.columns, 
)

Seeing as first_review and review_scores_rating have the exact same amount of missing values, it may be safe to assume that this was a feature that was added to the dataset at a later point, or these listings have never had a review

In [ ]:
airbnb.shape[0]-airbnb.shape[0]*0.173 # number of records if removed

In [ ]:
# Intermediate check of working dataset
pd.set_option('display.max_rows', 10)
airbnb.sort_values("review_scores_rating", ascending=False)

In [ ]:
airbnb[airbnb["review_scores_rating"].isna()]

Upon further inspection it seems the latter, and we can remove these values in order to work only with rated listings.

In [ ]:
# Remove remaining missing records
airbnb.dropna(inplace=True)

In [ ]:
airbnb.shape

### Handling Outliers

In [ ]:
# Summary statistics of the quantitative features of the dataset
airbnb.drop(['id', 'host_id'], axis=1).describe()

In [ ]:
plt.boxplot(airbnb['calculated_host_listings_count'])
plt.title("Boxplot of Host Listings")
plt.ylabel("Listings")
plt.xlabel("calculated_host_listings_count")
plt.xticks([])
plt.show()

There are a couple hosts with an extreme number of listings (good for them!), their number of listings may have influence on the rest of their listing attributes, and as such we will remove them. The goal in doing this is so that the summary statistics will be more robust and representative of the majority of the data. Potentially removing any leveraging bias for any models we decide to build.

In [ ]:
airbnb.groupby("host_id")['calculated_host_listings_count'].aggregate("max").sort_values(ascending=False)

The two hosts have over double the amount of listings as the host with the third highest amount of listings. Quite amazing, however not for our analyses.

In [ ]:
outliers = airbnb.groupby("host_id")['calculated_host_listings_count'].aggregate("max").nlargest(2).index
for host_id in outliers:
    print(host_id)

In [ ]:
airbnb.drop(airbnb[airbnb['host_id'==227662329]].index)

# Analysis

Analyze the data to gain a better understanding of it and locate any emerging trends.

In [ ]:
# Summary statistics of the quantitative features of the dataset
airbnb.drop(['id', 'host_id'], axis=1).describe()

Distribution of room types.

In [ ]:
airbnb.room_type.unique()

In [ ]:
room_types = airbnb.groupby("room_type")["id"].aggregate("count")
plt.barh(room_types.index, room_types, color="green", height=0.3)
plt.title("Number of Airbnbs by Room Type in Vancouver")
plt.xlabel("Frequency")
plt.ylabel("Type")
room_types

In [ ]:
plt.hist(airbnb["review_scores_rating"])
plt.title('Airbnb Ratings in Vancouver')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()

# Visualization

Plot findings on a read-world map using latitude and longitude coordinates with the Folium module.

In [4]:
# map_data = airbnb.sort_values("review_scores_value", ascending=False).head()
map_data = airbnb_raw.head(50)
map_figure = folium.Map([map_data.latitude.mean(), map_data.longitude.mean()], zoom_start=12)

def html_popup(row):
    name = map_data['name'].iloc[row]
    neighbourhood = map_data['neighbourhood_cleansed'].iloc[row]
    rate = map_data['review_scores_rating'].iloc[row]
    url = map_data['listing_url'].iloc[row]
    price = map_data['price'].iloc[row]
    min_nights = map_data['minimum_nights'].iloc[row]
    accommodates = map_data['accommodates'].iloc[row]
    lat = map_data['latitude'].iloc[row]
    lon = map_data['longitude'].iloc[row]
    scraped = map_data['last_scraped'].iloc[row]
    
    html = f"""
    <div style="width: 20em; line-height: 0.5em;">
        <h4 style="color: darkred;">{name}</h4>
        <h5>{neighbourhood}</h5>
        <h6>{str(rate)} Stars</h6>
        <h6>{str(price)} / Night</h6>
        <p>{min_nights} Nights Minimum</p>
        <p>Accommodates {accommodates}</p>
        <br>
        <div style="text-align: right;">
            <a href={url}>{url}</a>
            <p>Last updated {scraped}</p>
        </div>
    </div>
    """
    return html

for index, listing in map_data.iterrows():
    folium.Marker([listing['latitude'], listing['longitude']],
                        popup=html_popup(index),
                        icon = folium.Icon(icon="house", prefix="fa", color="red"),
                        tooltip = listing["name"],
                       ).add_to(map_figure)

map_figure

In [5]:
map_figure.save('../results/index.html')